<a href="https://colab.research.google.com/github/eman-kom/tensorflow-fundamentals/blob/main/08_NLP_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d39c8846-f976-e6f1-4ee5-af2d22270eaa)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-03 11:43:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-03 11:43:18 (95.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-07-03 11:43:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.128, 74.125.24.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-03 11:43:20 (152 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "<real disaster>" if target > 0 else "<not a real disaster>")
  print(f"Text\n{text}\n")
  print("---\n")

Target: 1 <real disaster>
Text
Heat Advisory is In Effect From 1 PM Through 7 PM Thursday. A Building Heat Wave and Increasing Humidity... #lawx http://t.co/u0SYkowVWV

---

Target: 1 <real disaster>
Text
Moderate #drought is spreading rapidly across central Ga. #Macon #WarnerRobins #Dublin #Milledgeville #gawx http://t.co/PHNEZ60cwe

---

Target: 0 <not a real disaster>
Text
fatal attraction

---

Target: 1 <real disaster>
Text
@HassanRouhani Wars doomed to destruction loss money must invest in Iran's inside that should not go outside

---

Target: 0 <not a real disaster>
Text
U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/MatIJwkzbh via @Change

---



In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [12]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=15,
)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
text_vectorizer(["There's a flood in my street!"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [16]:
random_sent = random.choice(train_sentences)
print(f"Before: {random_sent}")
text_vectorizer([random_sent])

Before: Alhaji Putin is far from being a good person sha. At least I had front row seat to his complete obliteration of Ibeto cement a competitor.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 2013,    9,  685,   20,  121,    3,  136,  674, 8617,   17,
         506,    8,   94,  851]])>

In [17]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [18]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=10000,
    output_dim=128,
    input_length=15
)

embedding

In [19]:
sample_embed = embedding(text_vectorizer([random.choice(train_sentences)]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03727751, -0.02689394,  0.0271987 , ...,  0.02943296,
         -0.02965199, -0.04039682],
        [-0.04010807, -0.03856973,  0.00970884, ...,  0.01869459,
         -0.02255371, -0.0112872 ],
        [ 0.02615496,  0.03963729,  0.02545469, ..., -0.0017709 ,
         -0.01734551,  0.03003231],
        ...,
        [ 0.02722417,  0.00635195, -0.04712627, ...,  0.01524002,
         -0.02061172,  0.0261586 ],
        [ 0.02429641, -0.0047024 , -0.0088992 , ..., -0.03763117,
         -0.00627802, -0.00820918],
        [-0.029166  ,  0.03177846, -0.00140629, ..., -0.04731046,
         -0.00186387, -0.02433319]]], dtype=float32)>

Model 0: Baseline Model -> Sklearn Multinomial Naive Bayes using TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [21]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 79.27%


In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [23]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:5]

array([1, 1, 1, 0, 0])

In [24]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [25]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

Model 1: Simple Dense Model

In [26]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [27]:
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [28]:
model_1_history = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="model_1_dense")]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20220703-114326
Epoch 1/5
215/215 [==============================] - 7s 9ms/step - loss: 0.6112 - accuracy: 0.6948 - val_loss: 0.5379 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.4406 - accuracy: 0.8171 - val_loss: 0.4708 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3468 - accuracy: 0.8612 - val_loss: 0.4581 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2839 - accuracy: 0.8914 - val_loss: 0.4642 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2369 - accuracy: 0.9140 - val_loss: 0.4813 - val_accuracy: 0.7795


In [29]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4813 - accuracy: 0.7795


[0.48125478625297546, 0.7795275449752808]

In [30]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[4.45871383e-01],
       [8.42670619e-01],
       [9.97585535e-01],
       [1.16522722e-01],
       [1.24890082e-01],
       [9.46157396e-01],
       [9.22158778e-01],
       [9.92941022e-01],
       [9.72038090e-01],
       [2.80785501e-01],
       [1.23290546e-01],
       [7.12918043e-01],
       [6.85114861e-02],
       [1.93636253e-01],
       [5.17287385e-03],
       [1.70805246e-01],
       [3.17423008e-02],
       [8.33095983e-02],
       [2.79326290e-01],
       [6.12702310e-01],
       [9.21708167e-01],
       [5.15849069e-02],
       [4.70238239e-01],
       [1.00295469e-01],
       [9.63445365e-01],
       [9.98654604e-01],
       [3.34014632e-02],
       [8.70407298e-02],
       [3.79297063e-02],
       [2.11682558e-01],
       [6.01301670e-01],
       [2.43534237e-01],
       [4.99146312e-01],
       [1.86873153e-01],
       [5.21638811e-01],
       [6.53013736e-02],
       [9.94547606e-01],
       [1.53341562e-01],
       [3.58494595e-02],
       [9.98688281e-01],


In [31]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [32]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results, baseline_results

({'accuracy': 77.95275590551181,
  'f1': 0.7771404562571971,
  'precision': 0.7822644211580037,
  'recall': 0.7795275590551181},
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

In [33]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

Model 2: LSTM Model

In [34]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [35]:
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [36]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220703-114349
Epoch 1/5
215/215 [==============================] - 8s 11ms/step - loss: 0.2233 - accuracy: 0.9245 - val_loss: 0.6062 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1598 - accuracy: 0.9403 - val_loss: 0.6380 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1286 - accuracy: 0.9510 - val_loss: 0.6874 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1056 - accuracy: 0.9616 - val_loss: 0.7113 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0855 - accuracy: 0.9660 - val_loss: 1.0618 - val_accuracy: 0.7677


In [37]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[2.4585113e-02],
       [6.4015108e-01],
       [9.9996161e-01],
       [8.8868812e-03],
       [6.7528585e-05],
       [9.9985290e-01],
       [8.8372177e-01],
       [9.9997890e-01],
       [9.9996459e-01],
       [2.8283694e-01]], dtype=float32)

In [38]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [39]:
model_2_results = calculate_results(y_true=val_labels, y_pred=model_2_preds)
model_2_results

{'accuracy': 76.77165354330708,
 'f1': 0.7652718344725522,
 'precision': 0.769927879249698,
 'recall': 0.7677165354330708}

Model 3: GRU Model

In [40]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GRU(64, return_sequences=True)(x)
#x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [41]:
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [42]:
model_3_history = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_GRU")]
)

Saving TensorBoard log files to: model_logs/model_2_GRU/20220703-114404
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1567 - accuracy: 0.9406 - val_loss: 0.7696 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0854 - accuracy: 0.9704 - val_loss: 0.8460 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0701 - accuracy: 0.9721 - val_loss: 0.9926 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0585 - accuracy: 0.9750 - val_loss: 1.0867 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0524 - accuracy: 0.9769 - val_loss: 1.1515 - val_accuracy: 0.7717


In [43]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.3164801e-03],
       [7.9019982e-01],
       [9.9986756e-01],
       [1.0751158e-01],
       [1.4357111e-05],
       [9.9933487e-01],
       [6.5583456e-01],
       [9.9993455e-01],
       [9.9986339e-01],
       [4.5286149e-01]], dtype=float32)

In [44]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_3_results = calculate_results(y_true=val_labels, y_pred=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'f1': 0.7684419786377588,
 'precision': 0.7758922411539364,
 'recall': 0.7716535433070866}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

Model 4: Bi-Directional RNN Model

In [50]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [51]:
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [53]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_GRU")]
)

Saving TensorBoard log files to: model_logs/model_4_GRU/20220703-115103
Epoch 1/5
215/215 [==============================] - 3s 11ms/step - loss: 0.1056 - accuracy: 0.9698 - val_loss: 1.1664 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0473 - accuracy: 0.9788 - val_loss: 1.6049 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0448 - accuracy: 0.9794 - val_loss: 1.5765 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0418 - accuracy: 0.9801 - val_loss: 1.5602 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0384 - accuracy: 0.9793 - val_loss: 1.7113 - val_accuracy: 0.7717


In [54]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[2.0426762e-04],
       [9.1666454e-01],
       [9.9999976e-01],
       [2.1038553e-01],
       [2.7362200e-06],
       [9.9999559e-01],
       [9.9486226e-01],
       [1.0000000e+00],
       [9.9999976e-01],
       [9.5442921e-01]], dtype=float32)

In [55]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [56]:
model_4_results = calculate_results(y_true=val_labels, y_pred=model_4_preds)
model_4_results

{'accuracy': 77.16535433070865,
 'f1': 0.7695827090439606,
 'precision': 0.7732961359962456,
 'recall': 0.7716535433070866}